## FEATURE IMPORTANCE & SELECTION

CONTEXT 

with 400+ features, not all contribute meaningfully to predictions. this will be a baseline model to estimate feature importance, enabling informed decision about which features to keep , drop , or engginer further